In [1]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

In [2]:
# turn off/on code for the result HTML page

display(Markdown('*For the result HTML page:* '))
    
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

*For the result HTML page:* 

In [3]:
##Importing pyodbc module
import turbodbc

In [4]:
connection = turbodbc.connect(dsn='FileCatalog')

In [5]:
connection

In [6]:
cursor = connection.cursor()

In [7]:
cursor.execute('SELECT * from datasets limit 5')

In [8]:
cursor.fetchall()

[['DST_TRUTH_G4HIT_pythia8_pp_mb_3MHz-0000000003-02374.root',
  3,
  2374,
  11044833070,
  'mdc2',
  'DST_TRUTH_G4HIT',
  1000],
 ['DST_TRUTH_G4HIT_pythia8_pp_mb_3MHz-0000000003-03606.root',
  3,
  3606,
  10952759042,
  'mdc2',
  'DST_TRUTH_G4HIT',
  1000],
 ['DST_TRKR_HIT_pythia8_Charm_3MHz-0000000004-39847.root',
  4,
  39847,
  2545997479,
  'mdc2',
  'DST_TRKR_HIT',
  819],
 ['G4Hits_pythia8_Jet04-0000000004-24479.root',
  4,
  24479,
  2997488460,
  'mdc2',
  'G4Hits',
  1000],
 ['DST_TRKR_G4HIT_pythia8_pp_mb_3MHz-0000000003-05560.root',
  3,
  5560,
  11035185852,
  'mdc2',
  'DST_TRKR_G4HIT',
  1000]]

In [9]:
import pyodbc
import pandas as pd
pd.set_option('display.width', 1000)

In [10]:
cnxn = pyodbc.connect('DSN=FileCatalog')
sql = "SELECT * from datasets limit 5"
data = pd.read_sql(sql,cnxn)  # without parameters [non-prepared statement]

# with a prepared statement, use list/tuple/dictionary of parameters depending on DB
#data = pd.read_sql(sql=sql, con=cnxn, params=query_params) 

In [11]:
data

,filename,runnumber,segment,size,dataset,dsttype,events
0,DST_TRUTH_G4HIT_pythia8_pp_mb_3MHz-0000000003-...,3,2374,11044833070,mdc2,DST_TRUTH_G4HIT,1000
1,DST_TRUTH_G4HIT_pythia8_pp_mb_3MHz-0000000003-...,3,3606,10952759042,mdc2,DST_TRUTH_G4HIT,1000
2,DST_TRKR_HIT_pythia8_Charm_3MHz-0000000004-398...,4,39847,2545997479,mdc2,DST_TRKR_HIT,819
3,G4Hits_pythia8_Jet04-0000000004-24479.root,4,24479,2997488460,mdc2,G4Hits,1000
4,DST_TRKR_G4HIT_pythia8_pp_mb_3MHz-0000000003-0...,3,5560,11035185852,mdc2,DST_TRKR_G4HIT,1000


In [12]:
unqiue_DSTs = pd.read_sql("SELECT DISTINCT runnumber, dataset, dsttype, substring(substring(filename, 1, POSITION('-' in filename)-1), LENGTH(dsttype)+2) as prodtype  from datasets order by dataset, runnumber, dsttype ASC",cnxn)

In [13]:
unqiue_DSTs

,runnumber,dataset,dsttype,prodtype
0,3,mdc2,DST_BBC_G4HIT,pythia8_Bottom_3MHz
1,3,mdc2,DST_BBC_G4HIT,pythia8_pp_mb_3MHz
2,3,mdc2,DST_BBC_G4HIT,sHijing_0_20fm_25kHz_bkg_0_20fm
3,3,mdc2,DST_CALO_CLUSTER,pythia8_Bottom_3MHz
4,3,mdc2,DST_CALO_CLUSTER,sHijing_0_20fm
...,...,...,...,...
76,4,mdc2,DST_VERTEX,sHijing_0_20fm_50kHz_bkg_0_20fm
77,4,mdc2,G4Hits,pythia8_Charm
78,4,mdc2,G4Hits,pythia8_Jet04
79,4,mdc2,G4Hits,pythia8_pp_mb


In [14]:
SampleData = pd.read_sql("SELECT substring(substring(filename, 1, POSITION('-' in filename)-1), LENGTH(dsttype)+2) as prodtype from datasets limit 5",cnxn)
SampleData

,prodtype
0,pythia8_pp_mb_3MHz
1,pythia8_pp_mb_3MHz
2,pythia8_Charm_3MHz
3,pythia8_Jet04
4,pythia8_pp_mb_3MHz


In [15]:
unqiue_DSTs = pd.read_sql("SELECT runnumber, dataset, dsttype, substring(substring(filename, 1, POSITION('-' in filename)-1), LENGTH(dsttype)+2) as prodtype, sum(events) , sum(size), COUNT(filename) from datasets group by runnumber, dataset, dsttype, prodtype",cnxn)

In [16]:
unqiue_DSTs

,runnumber,dataset,dsttype,prodtype,sum,sum,count
0,3,mdc2,DST_BBC_G4HIT,pythia8_Bottom_3MHz,8468743,3.893528e+12,10000
1,3,mdc2,DST_BBC_G4HIT,pythia8_pp_mb_3MHz,9971000,4.571586e+12,9971
2,3,mdc2,DST_BBC_G4HIT,sHijing_0_20fm_25kHz_bkg_0_20fm,8156,3.818717e+09,20
3,3,mdc2,DST_CALO_CLUSTER,pythia8_Bottom_3MHz,8468743,9.935386e+11,10000
4,3,mdc2,DST_CALO_CLUSTER,sHijing_0_20fm,10000,1.309836e+10,20
...,...,...,...,...,...,...,...
76,4,mdc2,DST_VERTEX,sHijing_0_20fm_50kHz_bkg_0_20fm,11967880,1.270971e+10,29295
77,4,mdc2,G4Hits,pythia8_Charm,60000000,5.050616e+13,60000
78,4,mdc2,G4Hits,pythia8_Jet04,39999000,1.199075e+14,39999
79,4,mdc2,G4Hits,pythia8_pp_mb,10000000,3.877923e+12,10000


In [17]:
unqiue_DSTs.loc[unqiue_DSTs['prodtype'] == 'pythia8_Charm_3MHz']

,runnumber,dataset,dsttype,prodtype,sum,sum,count
30,4,mdc2,DST_BBC_G4HIT,pythia8_Charm_3MHz,50815325,2.358871e+13,60000
36,4,mdc2,DST_CALO_CLUSTER,pythia8_Charm_3MHz,50811101,5.199879e+12,59995
40,4,mdc2,DST_CALO_G4HIT,pythia8_Charm_3MHz,50815325,3.227454e+14,60000
47,4,mdc2,DST_TRACKS,pythia8_Charm_3MHz,50685438,1.559750e+14,59847
51,4,mdc2,DST_TRKR_G4HIT,pythia8_Charm_3MHz,50815325,5.498944e+14,60000
57,4,mdc2,DST_TRKR_HIT,pythia8_Charm_3MHz,50740463,1.568381e+14,59912
61,4,mdc2,DST_TRUTH,pythia8_Charm_3MHz,50740463,7.385189e+14,59912
65,4,mdc2,DST_TRUTH_G4HIT,pythia8_Charm_3MHz,4636765,5.153375e+13,5473
71,4,mdc2,DST_VERTEX,pythia8_Charm_3MHz,50815325,2.683060e+10,60000


In [18]:

data = pd.read_sql( "SELECT * from datasets where dsttype='DST_TRUTH' and runnumber = 4 and segment < 10 limit 5 ",cnxn)  
data

,filename,runnumber,segment,size,dataset,dsttype,events
0,DST_TRUTH_pythia8_pp_mb_3MHz-0000000004-00000....,4,0,12496994862,mdc2,DST_TRUTH,875
1,DST_TRUTH_pythia8_pp_mb_3MHz-0000000004-00005....,4,5,11935779569,mdc2,DST_TRUTH,835
2,DST_TRUTH_pythia8_pp_mb_3MHz-0000000004-00006....,4,6,11951556326,mdc2,DST_TRUTH,839
3,DST_TRUTH_pythia8_pp_mb_3MHz-0000000004-00009....,4,9,12182785170,mdc2,DST_TRUTH,857
4,DST_TRUTH_pythia8_pp_mb_3MHz-0000000004-00002....,4,2,12165670786,mdc2,DST_TRUTH,848


In [19]:
data['filename'].loc[0]

'DST_TRUTH_pythia8_pp_mb_3MHz-0000000004-00000.root'

In [22]:

data = pd.read_sql( "SELECT * from datasets where runnumber = 4 and segment =0 and filename like '%pythia8_Charm_3MHz%'",cnxn)  
data

,filename,runnumber,segment,size,dataset,dsttype,events
0,DST_CALO_CLUSTER_pythia8_Charm_3MHz-0000000004...,4,0,85814399,mdc2,DST_CALO_CLUSTER,841
1,DST_BBC_G4HIT_pythia8_Charm_3MHz-0000000004-00...,4,0,391891753,mdc2,DST_BBC_G4HIT,841
2,DST_CALO_G4HIT_pythia8_Charm_3MHz-0000000004-0...,4,0,5361931902,mdc2,DST_CALO_G4HIT,841
3,DST_TRKR_G4HIT_pythia8_Charm_3MHz-0000000004-0...,4,0,9139466644,mdc2,DST_TRKR_G4HIT,841
4,DST_TRUTH_G4HIT_pythia8_Charm_3MHz-0000000004-...,4,0,9384503904,mdc2,DST_TRUTH_G4HIT,841
5,DST_VERTEX_pythia8_Charm_3MHz-0000000004-00000...,4,0,445615,mdc2,DST_VERTEX,841
6,DST_TRACKS_pythia8_Charm_3MHz-0000000004-00000...,4,0,2611654224,mdc2,DST_TRACKS,841
7,DST_TRKR_HIT_pythia8_Charm_3MHz-0000000004-000...,4,0,2621341131,mdc2,DST_TRKR_HIT,841
8,DST_TRUTH_pythia8_Charm_3MHz-0000000004-00000....,4,0,12301773551,mdc2,DST_TRUTH,841


In [23]:
data['filename'].to_numpy()

array(['DST_CALO_CLUSTER_pythia8_Charm_3MHz-0000000004-00000.root',
       'DST_BBC_G4HIT_pythia8_Charm_3MHz-0000000004-00000.root',
       'DST_CALO_G4HIT_pythia8_Charm_3MHz-0000000004-00000.root',
       'DST_TRKR_G4HIT_pythia8_Charm_3MHz-0000000004-00000.root',
       'DST_TRUTH_G4HIT_pythia8_Charm_3MHz-0000000004-00000.root',
       'DST_VERTEX_pythia8_Charm_3MHz-0000000004-00000.root',
       'DST_TRACKS_pythia8_Charm_3MHz-0000000004-00000.root',
       'DST_TRKR_HIT_pythia8_Charm_3MHz-0000000004-00000.root',
       'DST_TRUTH_pythia8_Charm_3MHz-0000000004-00000.root'], dtype=object)